In [169]:
# %%bash
set -e
sudo apt-get update -y
sudo apt-get install -y git curl gnupg software-properties-common unzip python3-pip
echo "✅  Base packages installed"


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://apt.releases.hashicorp.com jammy InRelease                       
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
             Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 12%Reading package lists... 30%Reading package lists... 45%Reading package lists... 45%Reading package lists... 46%Reading package lists... 46%Reading package lists... 46%Reading package lists... 55%Reading package lists... 55%Reading package lists... 61%Reading package lists... 68%Reading package lists... 68%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 

In [170]:
# %%bash
set -e
if ! command -v terraform &>/dev/null; then
  curl -fsSL https://apt.releases.hashicorp.com/gpg | \
       sudo gpg --dearmor -o /usr/share/keyrings/hashicorp.gpg
  echo "deb [signed-by=/usr/share/keyrings/hashicorp.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main" | \
       sudo tee /etc/apt/sources.list.d/hashicorp.list > /dev/null
  sudo apt-get update -y
  sudo apt-get install -y terraform
fi
terraform -version


Terraform v1.11.4
on linux_amd64


In [171]:
# %%bash
# ─── EDIT THESE FOUR LINES ────────────────────────────────────────────────────
export GIT_REPO="https://github.com/Akkey01/MLops-Final-Project.git"   # ◀︎ EDIT
export CLONE_DIR="/work/MLops-Final-Project"                           # ◀︎ EDIT
export OPENRC_FILE="/home/ng3230_nyu_edu/work/MLops-Final-Project/infrastructure/tf/kvm/openrc.sh"                             # ◀︎ EDIT
export TF_SUFFIX="team39"                                              # ◀︎ EDIT
export TF_KEYPAIR="id_rsa_chameleon"
# ──────────────────────────────────────────────────────────────────────────────
echo "Variables set:"
printf " GIT_REPO     = %s\n CLONE_DIR   = %s\n OPENRC_FILE = %s\n TF_SUFFIX   = %s\n TF_KEYPAIR  = %s\n" \
       "$GIT_REPO" "$CLONE_DIR" "$OPENRC_FILE" "$TF_SUFFIX" "$TF_KEYPAIR"


Variables set:
 GIT_REPO     = https://github.com/Akkey01/MLops-Final-Project.git
 CLONE_DIR   = /work/MLops-Final-Project
 OPENRC_FILE = /home/ng3230_nyu_edu/work/MLops-Final-Project/infrastructure/tf/kvm/openrc.sh
 TF_SUFFIX   = team39
 TF_KEYPAIR  = id_rsa_chameleon


In [172]:
sudo apt-get install -y git python3-pip
pip install --user --upgrade python-openstackclient


Reading package lists... 0%Reading package lists... 0%Reading package lists... 0%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 4%Reading package lists... 10%Reading package lists... 27%Reading package lists... 42%Reading package lists... 45%Reading package lists... 45%Reading package lists... 46%Reading package lists... 46%Reading package lists... 55%Reading package lists... 55%Reading package lists... 55%Reading package lists... 68%Reading package lists... 68%Reading package lists... 72%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 73%Reading package lists... 82%Reading package lists... 82%Reading package lists... 94%Reading package lists... 94%Reading package lists... 96%Reading package lists... 98%Reading package lists... 98%Reading package lists... 98

In [173]:
# %%bash
set -e
pip install --user --upgrade --no-cache-dir python-openstackclient
if ! grep -q '.local/bin' ~/.bashrc; then
  echo 'export PATH=$HOME/.local/bin:$PATH' >> ~/.bashrc
fi
export PATH=$HOME/.local/bin:$PATH
openstack --version


openstack 8.0.0


In [174]:
# %%bash
set -e
if [ ! -d "$CLONE_DIR/.git" ]; then
  git clone --recurse-submodules "$GIT_REPO" "$CLONE_DIR"
else
  echo "Repo exists; doing git pull ..."
  git -C "$CLONE_DIR" pull --ff-only
fi


Repo exists; doing git pull ...
Already up to date.


In [175]:
# %%bash
set -e
if [ ! -f "$OPENRC_FILE" ]; then
  echo "ERROR: $OPENRC_FILE not found" >&2
  exit 1
fi
unset $(env | grep -oE '^OS_[A-Z_]+')    # clean old vars
source "$OPENRC_FILE"
openstack token issue >/dev/null && echo "✅  Auth OK"


✅  Auth OK


In [176]:
# Delete Instance

# ls
# cd /work/MLops-Final-Project/infrastructure/tf/kvm
# terraform destroy -auto-approve
# terraform state list
# terraform state rm \
#   'openstack_compute_instance_v2.nodes["node1"]' \
#   'openstack_compute_instance_v2.nodes["node2"]' \
#   'openstack_compute_instance_v2.nodes["node3"]'
# |



In [177]:
# %%bash
set -e
cd "$CLONE_DIR/infrastructure/tf/kvm"

# generate a local tfvars file so commands stay short
cat > local.auto.tfvars <<EOF
suffix = "${TF_SUFFIX}"
key    = "${TF_KEYPAIR}"
EOF

terraform init -input=false
terraform plan -out apply.plan
terraform apply -auto-approve apply.plan

# 3) capture just the one Floating IP
floating_ip=$(terraform output -raw floating_ip_out)

# 4) print it (and also save to a file if you want)
echo "🎉 Terraform finished. Floating IP: $floating_ip"
echo "$floating_ip" > last_floating_ip.txt


# %%bash
CONTROL_IP=$floating_ip           # ← leave as is
SSH_KEY=~/.ssh/id_rsa

cat > /work/MLops-Final-Project/infrastructure/ansible/inventory.ini <<EOF
[control]
node1 ansible_host=${CONTROL_IP}

[workers]
node2 ansible_host=192.168.1.12
node3 ansible_host=192.168.1.13

[all:vars]
ansible_user=cc
ansible_ssh_private_key_file=${SSH_KEY}
ansible_ssh_common_args='-o StrictHostKeyChecking=no'

[workers:vars]
ansible_ssh_common_args='-o StrictHostKeyChecking=no -o ProxyCommand="ssh -i ${SSH_KEY} -o StrictHostKeyChecking=no -W %h:%p cc@${CONTROL_IP}"'
EOF

cat /work/MLops-Final-Project/infrastructure/ansible/inventory.ini

# %%bash
ANSIBLE_DIR=/work/MLops-Final-Project/infrastructure/ansible
SSH_KEY=~/.ssh/id_rsa

mkdir -p "$ANSIBLE_DIR"

cat > "$ANSIBLE_DIR/ansible.cfg" <<EOF
[defaults]
inventory            = ./inventory.ini
remote_user          = cc
private_key_file     = ~/.ssh/id_rsa
host_key_checking    = False
retry_files_enabled  = False
stdout_callback      = yaml
any_errors_fatal     = False

[ssh_connection]
ssh_args = -o StrictHostKeyChecking=off \
           -o UserKnownHostsFile=/dev/null \
           -o ForwardAgent=yes \
           -o ProxyCommand="ssh -o StrictHostKeyChecking=no \
                             -o UserKnownHostsFile=/dev/null \
                             -W %h:%p cc@${floating_ip}"
pipelining = True
EOF

echo "Written $ANSIBLE_DIR/ansible.cfg:"
cat "$ANSIBLE_DIR/ansible.cfg"


echo "Installing python3-six, python3-setuptools on all nodes …"
ansible -i $ANSIBLE_DIR/inventory.ini all \
  -m raw \
  -a 'sudo DEBIAN_FRONTEND=noninteractive apt-get update -y &&
      sudo DEBIAN_FRONTEND=noninteractive apt-get install -y \
      python3 python3-apt python3-pip python3-six python3-setuptools'\
  --become -vv

# 1) Remove the apt copy (keeps only python libs on the targets)
sudo apt-get remove -y ansible-core

# 2) Install 2.15.x in your user env (control plane only)
pip install --user --upgrade "ansible-core<2.16"   # installs 2.15.5 today
export PATH=$HOME/.local/bin:$PATH
ansible --version    # should say 2.15.x

export ANSIBLE_CONFIG=/work/MLops-Final-Project/infrastructure/ansible/ansible.cfg
cd /work/MLops-Final-Project/infrastructure/ansible

# fast check
ansible all -i inventory.ini -m ping

# full run
ansible-playbook -i inventory.yml general/hello_host.yml

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of terraform-provider-openstack/openstack from the dependency lock file
- Using previously-installed terraform-provider-openstack/openstack v1.51.1

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_secgroup_v2.allow_8000: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack_netwo